# Monthly KPI April 2023 number check

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

In [3]:
sq = """WITH
  dt AS (
  SELECT
    DATE('2023-04-01') WkstartDate,
    DATE('2023-04-30') WkendDate),
disb as (select distinct disbursementDateTime, customerId, loanType, loanAccountNumber, SUBSTR(mobileNo, -10) as mobileno , loanPaidStatus
, row_number() over(partition by SUBSTR(mobileNo, -10) order by SUBSTR(mobileNo, -10))rnk
from `risk_credit_mis.loan_master_table`, dt 
where 
date_trunc(disbursementDateTime, day) <= dt.WkendDate
-- between dt.WkstartDate and dt.WkendDate 
and 
coalesce(loanPaidStatus, 'NA') not in ('Settled','Completed')
)
,
wc1 as 
(
SELECT
  dt.WkstartDate,
  dt.WkendDate,
  Genesys_call_id,
  callDatetime,
  COALESCE(IS_APC, 0) APC,
  COALESCE(IS_RPC, 0) RPC,
  COALESCE(IS_PTP,0) PTP,
  RIGHT(mobileNumber, 10) mobileNumber,
  g.campaignName,
  g.agentGroup,
  g.agentFullName,
  g.employeeId,
  g.connected,
  g.notConnected,
  g.callResult,
  ROW_NUMBER() OVER(PARTITION BY Genesys_call_id ORDER BY callDatetime DESC, COALESCE(IS_APC, 0), COALESCE(IS_RPC, 0), COALESCE(IS_PTP, 0))rnk
FROM
  `risk_credit_mis.call_attempt_history_gensys` g,  dt
LEFT JOIN
  `dap_ds_poweruser_playground.campaignmaster` cm
ON
  UPPER(cm.Campaignname) = UPPER(g.campaignName)
INNER JOIN disb ON  SUBSTR(disb.mobileNo, -10) = SUBSTR(CAST(g.mobileNumber AS string), -10)
WHERE
  cm.reference = 'WC'
  and disb.rnk = 1
  and date(callDatetime) between  dt.WkstartDate and dt.WkendDate
)
, 
wc as 
(select wc1.WkstartDate, wc1.WkendDate, Genesys_call_id,
callDatetime,
APC,
RPC,
PTP,
case when APC = 1 then mobileNumber end apc_m,
case when RPC = 1 then mobileNumber end rpc_m,
case when PTP = 1 then mobileNumber end ptp_m,
mobileNumber,
campaignName,
agentGroup,
agentFullName,
employeeId,
connected,
notConnected,
callResult,
rnk,
from wc1 
where rnk = 1
)
select * from wc;"""

In [4]:
df = client.query(sq).to_dataframe()

In [5]:
df.head()

,WkstartDate,WkendDate,Genesys_call_id,callDatetime,APC,RPC,PTP,apc_m,rpc_m,ptp_m,mobileNumber,campaignName,agentGroup,agentFullName,employeeId,connected,notConnected,callResult,rnk
0,2023-04-01,2023-04-30,04a9e115-dd3a-4ab4-99ef-c715848b0ad1,2023-04-28 08:26:08.139,0,0,0,None,None,None,9456010288,PROD_Welcome call,None,None,None,0,1,Not Answered,1
1,2023-04-01,2023-04-30,05ec1362-bdbc-4e87-aecd-b3228b12f02f,2023-04-19 14:04:19.948,0,0,0,None,None,None,9458129894,PROD_Welcome call,None,None,None,0,1,Not Answered,1
2,2023-04-01,2023-04-30,06c3706d-1225-494f-bd86-a403a23c9abc,2023-04-04 16:31:31.385,0,0,0,None,None,None,9979521175,Prod_Welcome Call,None,None,None,0,1,Not Answered,1
3,2023-04-01,2023-04-30,07935b36-e06f-48c8-bba9-b54ade7cbade,2023-04-19 16:45:01.609,0,0,0,None,None,None,9088621773,PROD_Welcome call,None,None,None,0,1,Not Answered,1
4,2023-04-01,2023-04-30,08b098e6-e153-4cc1-80d0-6b0fe572b586,2023-04-24 09:18:51.414,0,0,0,None,None,None,9109881058,PROD_Welcome call,Collections,Robin Calingasan,562fe337-ce80-4071-bf03-44bd08f18514,1,0,Answered,1


In [6]:
df.to_csv(r"April2023_welcomecallcontactrate.csv", index = False)